In [1]:
%reset -f

# import packages

In [2]:
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import warnings

from sklearn.linear_model import LassoCV, LassoLarsCV, ElasticNetCV
from solar                import solar
from sklearn.exceptions   import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")

# Download prostate cancer data

In [3]:
prostate_table = pd.read_csv('https://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.data', sep = None)

/home/ning/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


# Check the data and remove unecessary variables

In [4]:
prostate_table.head()

,Unnamed: 0,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


In [5]:
prostate_table = prostate_table.drop(columns=['Unnamed: 0', 'train'])

prostate_table.to_csv(r'~/Dropbox/Working_Directory/Python/Stability/solar/application/Prostate.csv')

In [6]:
prostate_table.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564


# CV coordinate descend (CV-cd) for lasso regression

In [7]:
X_table = prostate_table.drop(columns=['lpsa'])
Y       = prostate_table['lpsa']
X       = np.array(X_table)
Y       = np.array(Y)

In [8]:
Y.shape = (97,)

In [9]:
reg2 = LassoCV(cv=10, random_state=0)
reg2.fit(X,Y)

lassoCV_ceof = reg2.coef_
lassoCV_ind  = np.nonzero(lassoCV_ceof)

# CV lars for lasso (CV-lars-lasso)

In [10]:
np.random.seed(0)

reg1 = LassoLarsCV(cv=10)
reg1.fit(X,Y)

lasso_lars_ceof = reg1.coef_
lasso_lars_ind  = np.nonzero(lasso_lars_ceof)

# solar regression

In [11]:
Y.shape = (97,1)

trial2 = solar( X, Y, 10, -0.02, lasso = False)
solar_coef, opt_c, test_error, Qc_list, _, _, _, _ = trial2.fit()

solar_ind = np.nonzero(solar_coef)[0]

trial2.q_list(Qc_list)

q_hat value >=  1.0
[['X0']]
q_hat value >=  0.8444444444444444
[['X0'], ['X1']]
q_hat value >=  0.6666666666666666
[['X0'], ['X1'], ['X6']]
q_hat value >=  0.4666666666666667
[['X0'], ['X1'], ['X6'], ['X3']]
q_hat value >=  0.37777777777777777
[['X0'], ['X1'], ['X6'], ['X3'], ['X7']]
q_hat value >=  0.35555555555555557
[['X0'], ['X1'], ['X6'], ['X3'], ['X7'], ['X4']]
q_hat value >=  0.24444444444444444
[['X0'], ['X1'], ['X6'], ['X3'], ['X7'], ['X4'], ['X2']]
q_hat value >=  0.044444444444444446
[['X0'], ['X1'], ['X6'], ['X3'], ['X7'], ['X4'], ['X2'], ['X5']]


# variables selected by solar

In [12]:
X_table.columns[solar_ind]

Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45'], dtype='object')

# variables selected by CV-lars-lasso

In [13]:
lasso_lars_active = X_table.columns[lasso_lars_ind]
print(np.count_nonzero(lasso_lars_ind))
print(lasso_lars_active)

7
Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45'], dtype='object')


# variables selected by CV-cd

In [14]:
lassoCV_active = X_table.columns[lassoCV_ind]
print(np.count_nonzero(lassoCV_ceof))
print(lassoCV_active)

7
Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'pgg45'], dtype='object')


# variables selected by CV elastic net (CV-en)

In [15]:
regr = ElasticNetCV(normalize=True, random_state=0)
regr.fit(X,Y)

en_coef   = regr.coef_
en_ind    = np.nonzero(en_coef)
en_active = X_table.columns[en_ind]
print(np.count_nonzero(en_coef))
print(en_active)

8
Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45'], dtype='object')


/home/ning/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:1100: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ning/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


# check the correlation table

In [16]:
prostate_table.corr(method='pearson')

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
lcavol,1.000000,0.280521,0.225000,0.027350,0.538845,0.675310,0.432417,0.433652,0.734460
lweight,0.280521,1.000000,0.347969,0.442264,0.155385,0.164537,0.056882,0.107354,0.433319
age,0.225000,0.347969,1.000000,0.350186,0.117658,0.127668,0.268892,0.276112,0.169593
lbph,0.027350,0.442264,0.350186,1.000000,-0.085843,-0.006999,0.077820,0.078460,0.179809
svi,0.538845,0.155385,0.117658,-0.085843,1.000000,0.673111,0.320412,0.457648,0.566218
lcp,0.675310,0.164537,0.127668,-0.006999,0.673111,1.000000,0.514830,0.631528,0.548813
gleason,0.432417,0.056882,0.268892,0.077820,0.320412,0.514830,1.000000,0.751905,0.368987
pgg45,0.433652,0.107354,0.276112,0.078460,0.457648,0.631528,0.751905,1.000000,0.422316
lpsa,0.734460,0.433319,0.169593,0.179809,0.566218,0.548813,0.368987,0.422316,1.000000


# check irrepresentable conditions for variable "gleason"

In [17]:
import statsmodels.api as sm

In [18]:
X_IRC_table = X_table[['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'pgg45']]
Y_IRC_table = X_table['gleason']

In [19]:
X_IRC_table =(X_IRC_table-X_IRC_table.mean())/X_IRC_table.std()
Y_IRC_table =(Y_IRC_table-Y_IRC_table.mean())/Y_IRC_table.std()

In [20]:
X_IRC_table = sm.add_constant(X_IRC_table)
OLS_reg = sm.OLS(Y_IRC_table, X_IRC_table)

OLS_result = OLS_reg.fit()
print(OLS_result.summary())

                            OLS Regression Results                            
Dep. Variable:                gleason   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                     18.68
Date:                Sat, 28 Sep 2019   Prob (F-statistic):           4.24e-15
Time:                        06:33:03   Log-Likelihood:                -93.302
No. Observations:                  97   AIC:                             202.6
Df Residuals:                      89   BIC:                             223.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.082e-17      0.067    3.1e-16      1.0

/home/ning/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [21]:
coef = OLS_result.params

In [22]:
np.array(coef)

array([ 2.08166817e-17,  1.72562421e-01, -9.15889904e-02,  7.01005780e-02,
        2.61197055e-02, -1.08026200e-01,  4.29329893e-02,  6.87824398e-01])

In [23]:
np.sum(np.abs(np.array(coef)))

1.19915528191927